# Convolutional neural networks (CNN) for CIFAR-10/100 using PyTorch

Markus Enzweiler, markus.enzweiler@hs-esslingen.de

This is a demo used in a Computer Vision & Machine Learning lecture. Feel free to use and contribute.

We build and train a CNN for CIFAR-10 / CIFAR-100 image classification, see https://www.cs.toronto.edu/~kriz/cifar.html. We use the Python code from https://github.com/menzHSE/torch-cifar-10-cnn.git and execute it via this notebook. 

## Setup

Adapt `packagePath` to point to the directory containing this notebeook.

In [ ]:
# Notebook id
nb_id = "cnn/cifar/torch"

# Imports
import sys
import os

In [ ]:
# Package Path (folder of this notebook)

#####################
# Local environment #
#####################

package_path = "./"


#########
# Colab #
#########


def check_for_colab():
    try:
        import google.colab

        return True
    except ImportError:
        return False


# running on Colab?
on_colab = check_for_colab()

if on_colab:
    # assume this notebook is run from Google Drive and the whole
    # cv-ml-lecture-notebooks repo has been setup via setupOnColab.ipynb

    # Google Drive mount point
    gdrive_mnt = "/content/drive"

    ##########################################################################
    # Ensure that this is the same as gdrive_repo_root in setupOnColab.ipynb #
    ##########################################################################
    # Path on Google Drive to the cv-ml-lecture-notebooks repo
    gdrive_repo_root = f"{gdrive_mnt}/MyDrive/cv-ml-lecture-notebooks"

    # mount drive
    from google.colab import drive

    drive.mount(gdrive_mnt, force_remount=True)

    # set package path
    package_path = f"{gdrive_repo_root}/{nb_id}"

# check whether package path exists
if not os.path.isdir(package_path):
    raise FileNotFoundError(f"Package path does not exist: {package_path}")

print(f"Package path: {package_path}")

In [ ]:
# Additional imports

# Repository Root
repo_root = os.path.abspath(os.path.join(package_path, "..", "..", ".."))
# Add the repository root to the system path
if repo_root not in sys.path:
    sys.path.append(repo_root)

# Package Imports
from nbutils import requirements as nb_reqs
from nbutils import colab as nb_colab
from nbutils import git as nb_git
from nbutils import exec as nb_exec
from nbutils import data as nb_data

In [ ]:
# Clone git repository

# Absolute path of the repository directory
repo_dir = os.path.join(package_path, "torch-cifar-10-cnn")
repo_url = "https://github.com/menzHSE/torch-cifar-10-cnn.git"

nb_git.clone(repo_url, repo_dir, on_colab)

In [ ]:
# Install requirements in the current Jupyter kernel
req_file = os.path.join(repo_dir, "requirements.txt")
nb_reqs.pip_install_reqs(req_file, on_colab)

# Train and test CNN on CIFAR-10



In [ ]:
# Let's see what we can do with train.py
nb_exec.executePythonScript(
    os.path.join(repo_dir, "train.py"), {"help": None}, on_colab
)

## Parameters

In [ ]:
# parameters
batchsize = 32
seed = 42
lr = 3e-4
epochs = 30
dataset = "CIFAR-10"

## Train

In [ ]:
params = {
    "dataset": dataset,  # dataset name
    "batchsize": batchsize,  # batch size
    "seed": seed,  # random seed
    "lr": lr,  # learning rate
    "epochs": epochs,  # number of epochs
}

# Execute 'train.py' with parameters
nb_exec.executePythonScript(os.path.join(repo_dir, "train.py"), params, on_colab)

## Test 

In [ ]:
# parameters
params = {"model": f"models/model_{dataset}_{epochs-1:03d}.pth"}  # model name

# Execute 'test.py' with parameters
nb_exec.executePythonScript(os.path.join(repo_dir, "test.py"), params, on_colab)

# Train and test CNN on CIFAR-100

## Parameters

In [ ]:
# parameters
batchsize = 32
seed = 42
lr = 3e-4
epochs = 30
dataset = "CIFAR-100"

## Train

In [ ]:
params = {
    "dataset": dataset,  # dataset name
    "batchsize": batchsize,  # batch size
    "seed": seed,  # random seed
    "lr": lr,  # learning rate
    "epochs": epochs,  # number of epochs
}

# Execute 'train.py' with parameters
nb_exec.executePythonScript(os.path.join(repo_dir, "train.py"), params, on_colab)

## Test

In [ ]:
# parameters
params = {
    "dataset": dataset,  # dataset name
    "model": f"models/model_{dataset}_{epochs-1:03d}.pth",  # model name
}

# Execute 'test.py' with parameters
nb_exec.executePythonScript(os.path.join(repo_dir, "test.py"), params, on_colab)

# Finetune a ResNet on CIFAR-100

## Parameters

In [ ]:
# parameters
batchsize = 32
seed = 42
lr = 3e-4
epochs = 5
dataset = "CIFAR-100"
finetune = "resnet18"

## Train

In [ ]:
params = {
    "dataset": dataset,  # dataset name
    "batchsize": batchsize,  # batch size
    "seed": seed,  # random seed
    "lr": lr,  # learning rate
    "epochs": epochs,  # number of epochs
    "finetune": finetune,  # finetune model
}

# Execute 'train.py' with parameters
nb_exec.executePythonScript(os.path.join(repo_dir, "train.py"), params, on_colab)

## Test

In [ ]:
# parameters
params = {
    "dataset": dataset,  # dataset name
    "model": f"models/model_{finetune}_{dataset}_{epochs-1:03d}.pth",  # model name
    "finetune": finetune,  # finetune model
}

# Execute 'test.py' with parameters
nb_exec.executePythonScript(os.path.join(repo_dir, "test.py"), params, on_colab)